<a href="https://colab.research.google.com/github/sameepshrestha/segmentation_state_of_the_art/blob/main/model_without_skip_layers_june23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==2.4.1
import tensorflow
print(tensorflow.__version__)

     |████████████████████████████████| 394.3MB 45kB/s 
     |████████████████████████████████| 3.8MB 43.1MB/s 
     |████████████████████████████████| 2.9MB 32.2MB/s 
     |████████████████████████████████| 471kB 56.6MB/s 
  Found existing installation: grpcio 1.34.1
    Uninstalling grpcio-1.34.1:
      Successfully uninstalled grpcio-1.34.1
  Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Found existing installation: tensorflow 2.5.0
    Uninstalling tensorflow-2.5.0:
      Successfully uninstalled tensorflow-2.5.0
2.4.1


In [ ]:
import tensorflow
print(tensorflow.__version__)

2.4.1


In [2]:
# model.summary()
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import matplotlib.pyplot as plt 
import numpy as np 
import cv2 as cv 
from PIL import Image, ImageDraw
import json 
import os 
import glob 
import re 
from tqdm import tqdm
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Conv2D, Reshape, Concatenate,MaxPooling2D, AveragePooling2D,Input, BatchNormalization, Activation, UpSampling2D, Concatenate, LeakyReLU,Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D,Multiply,SpatialDropout2D
from tensorflow.keras.optimizers import Adam 
#using tensorflow backend 16 so keras backend still 32bytes so no keras to be used
# from keras.layers.merge import concatenate,Concatenate
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import gc
import psutil

In [4]:
tf.keras.mixed_precision.experimental.set_policy('mixed_float16')
# # tf.compat.v1.disable_eager_execution()

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla T4, compute capability 7.5


In [5]:
category = """wall, floor, cabinet, bed, chair, sofa, table, door, window, bookshelf, picture, counter, blinds, desk, shelves, curtain, dresser
, pillow, mirror, floor mat, clothes, ceiling, books, fridge, tv, paper, towel, shower curtain, box, whiteboard, person, nightstand, toilet, sink
, lamp, bathtub, bag"""
category = category.split(r', ')
category = [i.replace('\n','') for i in category]
category_1 = ['background']
for categ in category:category_1.append(categ) 
print(category_1)
category_dict = {k:category_1.index(k) for k in category_1}
print(category_dict)
reverse_map={i:k for k,i in category_dict.items()}
color_list=['gray','red','green','#FFFF00','#8c564b','#4B8BBE','#306998','#FFE873','#FFD43B','#646464','#5a0000','#003a27','#C0C0C0','#808080','#800000','#808000','#00FF00','#00FFFF','#008080','#000080','#FF00FF','#800080','#CD5C5C','#F08080','#FA8072','#E9967A','#FFA07A','#DC143C','#FF7F50','#FFD700','#ffffe0','#bdb76b','#228b22','#B0E0E6','#4169e1','#f0ffff','#d2691e','#BC8F8F']
class_color_map={k:color_list[i] for i,k in enumerate(reverse_map.keys())}

['background', 'wall', 'floor', 'cabinet', 'bed', 'chair', 'sofa', 'table', 'door', 'window', 'bookshelf', 'picture', 'counter', 'blinds', 'desk', 'shelves', 'curtain', 'dresser', 'pillow', 'mirror', 'floor mat', 'clothes', 'ceiling', 'books', 'fridge', 'tv', 'paper', 'towel', 'shower curtain', 'box', 'whiteboard', 'person', 'nightstand', 'toilet', 'sink', 'lamp', 'bathtub', 'bag']
{'background': 0, 'wall': 1, 'floor': 2, 'cabinet': 3, 'bed': 4, 'chair': 5, 'sofa': 6, 'table': 7, 'door': 8, 'window': 9, 'bookshelf': 10, 'picture': 11, 'counter': 12, 'blinds': 13, 'desk': 14, 'shelves': 15, 'curtain': 16, 'dresser': 17, 'pillow': 18, 'mirror': 19, 'floor mat': 20, 'clothes': 21, 'ceiling': 22, 'books': 23, 'fridge': 24, 'tv': 25, 'paper': 26, 'towel': 27, 'shower curtain': 28, 'box': 29, 'whiteboard': 30, 'person': 31, 'nightstand': 32, 'toilet': 33, 'sink': 34, 'lamp': 35, 'bathtub': 36, 'bag': 37}


In [6]:
AUTO = tf.data.experimental.AUTOTUNE
Image_width = 640 
Image_height = 480 
no_classes = len(category_dict)
def load_data(path, split=0.2):
    images = sorted(glob.glob(os.path.join(path, "image/*")))
    print(len(images))
    masks = sorted(glob.glob(os.path.join(path, "mask2/*")))
    print(len(masks))
    total_size = 7990
    test_size = 1990
    valid_size=1000
    training_x, test_x = train_test_split(images[:7990], test_size=test_size, random_state=42)
    training_y, test_y = train_test_split(masks[:7990], test_size=test_size, random_state=42)
    train_x, valid_x = train_test_split(training_x, test_size=valid_size, random_state=42)
    train_y, valid_y = train_test_split(training_y, test_size=valid_size, random_state=42)
    return (train_x, train_y), (valid_x, valid_y),(test_x, test_y)
(train_x,train_y),(valid_x,valid_y),(test_x, test_y)=load_data('/content/drive/MyDrive/depth')
train_size=len(train_x)
valid_size=len(valid_x)

def read_image(path):
    path = path.decode()
    x = cv.imread(path, cv.IMREAD_COLOR)
    x = cv.resize(x,(640,480),interpolation=cv.INTER_AREA)
    x = x/255.0
    x=tf.cast(x,dtype=tf.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv.imread(path, cv.IMREAD_GRAYSCALE)
    x = cv.resize(x,(640,480),interpolation=cv.INTER_NEAREST)
  
    x = to_categorical(x,num_classes=no_classes)
    x = x[:,:,:]
    # x = np.expand_dims(x, axis=-1)
    # x=np.concatenate([x,x],axis=-1)
    x=tf.cast(x,dtype=tf.float32)
    return x
# read_mask('/content/drive/MyDrive/depth/mask100/mask1.png')
def parser(x,y):
    def _parse(x,y):
        x=read_image(x)
        y=read_mask(y)
        return x,y
    x,y = tf.numpy_function(_parse, [x,y], [tf.float32,tf.float32])
    x.set_shape([Image_height, Image_width, 3])
    y.set_shape([Image_height, Image_width, no_classes])
    return x,y
    
def tf_dataset(x, y, batch):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(parser)
    # dataset = dataset.cache()
    dataset = dataset.repeat()
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(AUTO)
    return dataset
train_dataset=tf_dataset(train_x,train_y,batch=16)
valid_dataset=tf_dataset(valid_x,valid_y,batch=16)
valid_dataset

8033
8033


<PrefetchDataset shapes: ((None, 480, 640, 3), (None, 480, 640, 38)), types: (tf.float32, tf.float32)>

In [ ]:
print(test_x[:100])

['/content/drive/MyDrive/depth/image/image5457.jpg', '/content/drive/MyDrive/depth/image/image5917.jpg', '/content/drive/MyDrive/depth/image/image7738.jpg', '/content/drive/MyDrive/depth/image/image7847.jpg', '/content/drive/MyDrive/depth/image/image3387.jpg', '/content/drive/MyDrive/depth/image/image516.jpg', '/content/drive/MyDrive/depth/image/image1978.jpg', '/content/drive/MyDrive/depth/image/image4452.jpg', '/content/drive/MyDrive/depth/image/image470.jpg', '/content/drive/MyDrive/depth/image/image2603.jpg', '/content/drive/MyDrive/depth/image/image3895.jpg', '/content/drive/MyDrive/depth/image/image5776.jpg', '/content/drive/MyDrive/depth/image/image164.jpg', '/content/drive/MyDrive/depth/image/image7588.jpg', '/content/drive/MyDrive/depth/image/image5720.jpg', '/content/drive/MyDrive/depth/image/image2827.jpg', '/content/drive/MyDrive/depth/image/image1903.jpg', '/content/drive/MyDrive/depth/image/image5203.jpg', '/content/drive/MyDrive/depth/image/image2773.jpg', '/content/driv

In [ ]:
#changes 
#Added a extra convolution in the encoder part lets look if it makes improvement or not 
def squeeze_excite_block(inputs, ratio=8):
    init = inputs
    filters = init.shape[-1]
    se_shape = (1, 1, filters)
    se = GlobalAveragePooling2D()(inputs)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)
    se = Multiply()([init, se])
    return se
    
def encoder_conv_block(inputs,filters):
  x = Conv2D(filters, (3, 3), padding="same",kernel_initializer='he_normal')(inputs)
  x = SpatialDropout2D(0.1)(x)
  x = BatchNormalization()(x)
  x = LeakyReLU(alpha=0.001)(x)
  x = Conv2D(filters, (3, 3), padding="same",kernel_initializer='he_normal')(x)
  x = SpatialDropout2D(0.1)(x)
  x = BatchNormalization()(x)
  x = LeakyReLU(alpha=0.001)(x)
#   x =  squeeze_excite_block(x)
  return x 
def decoder_conv_block(inputs,filters1,filters2):
  x = Conv2D(filters1, (3, 3), padding="same",kernel_initializer='he_normal')(inputs)
  x = SpatialDropout2D(0.1)(x)
  x = BatchNormalization()(x)
  x = LeakyReLU(alpha=0.001)(x)
  x = Conv2D(filters2, (3, 3), padding="same",kernel_initializer='he_normal')(x)
  x = SpatialDropout2D(0.1)(x)
  x = BatchNormalization()(x)
  x = LeakyReLU(alpha=0.001)(x)
  return x

def encoder(inputs):
  num_filters = [32,64, 128,256,256]
  x= inputs
  for i,filters  in enumerate(num_filters):
    x = encoder_conv_block(x,filters)
    x = MaxPooling2D((2, 2))(x)
  return x 
def decoder(x):
  num_filters = [32,64, 128,256,256,512]
  num_filters.reverse()
  for i in range(len(num_filters)-1):
    x = decoder_conv_block(x,num_filters[i],num_filters[i+1])
    x = UpSampling2D((2,2))(x)
    
  return x
def output_block(inputs,classes):
    x = Conv2D(classes,(1, 1), padding="same")(inputs)
    return x
def main_model(shape):
  inputs = Input(shape)
  encoded = encoder(inputs)
  decoded = decoder(encoded)
  output = output_block(decoded,classes=38)
  model = Model(inputs,output)
  return model

model = main_model((480,640,3))

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 480, 640, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 480, 640, 32)      896       
_________________________________________________________________
spatial_dropout2d (SpatialDr (None, 480, 640, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 480, 640, 32)      128       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 480, 640, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 480, 640, 32)      9248      
_________________________________________________________________
spatial_dropout2d_1 (Spatial (None, 480, 640, 32)      0     

In [ ]:
#june 18 5000 with an additional conv layer ..faster but not more accurate i guess

In [8]:
#loss function 

import gc 
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import CSVLogger ,LearningRateScheduler,Callback
# callbacks = [  
#     EarlyStopping(monitor='val_custom_metrics', patience=3),
#     ModelCheckpoint(filepath, monitor='val_custom_metrics',verbose=1,mode='max',save_best_only=True),
# ]
# loss = tf.keras.losses.categorical_crossentropy(from_logits=True)
def binary_loss(y_true, y_pred):
    loss = binary_crossentropy(y_true, y_pred,from_logits=True)
    loss_value = tf.math.reduce_mean(loss,axis=[0])
    return loss_value
# loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True, )
# def cross_entropy_loss(y_true,y_pred):
#     return loss(y_true,y_pred)
def cross_entropy_loss(y_true,y_pred):
    loss = tf.nn.softmax_cross_entropy_with_logits(
    y_true, y_pred, axis=-1, name=None)
    return loss

def custom_metrics(y_true,y_pred,smooth=1):
    y_pred = tf.math.argmax(y_pred, axis=-1)
    # y_pred = tf.expand_dims(y_pred,axis=-1)
    y_pred = tf.one_hot(y_pred,38)
    y_true = y_true[:,:,:,1:38]
    y_pred= y_pred[:,:,:,1:38]
    intersection  = tf.math.reduce_sum((y_true*y_pred),axis = [1,2,3])
    union = (tf.math.reduce_sum(y_true,axis=[1,2,3])+tf.math.reduce_sum(y_pred,axis = [1,2,3])) - intersection
    return tf.math.reduce_mean((intersection+smooth)/(union+smooth),axis=0)
smooth = 1
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)
model=tf.keras.models.load_model('/content/drive/MyDrive/new_architecture/model_normal-june22/',custom_objects={'cross_entropy_loss': cross_entropy_loss ,'dice_coef': dice_coef,'custom_metrics':custom_metrics})
model.compile(loss = cross_entropy_loss,optimizer=Adam(learning_rate =.001),metrics=[custom_metrics])

#31 percent accuracy with squeeze in encoder than overfits fast
for both decoder and encoder the accuracy was same as well so it may be bad to use before the downsampling


In [ ]:
epoch=30
# model.save('/content/drive/MyDrive/new_architecture/model_normal-june22/',save_format='tf')
t_steps=train_size//16
v_steps=valid_size//16
val_custom_metrics = 0.35
for i in range(epoch):
  print('epoch =',i)
  model.fit(train_dataset,epochs= 1,steps_per_epoch=t_steps)
  loss = model.evaluate(valid_dataset,steps=v_steps)
  if loss[1]>=val_custom_metrics:
    val_custom_metrics = loss[1]
    model.save('/content/drive/MyDrive/new_architecture/model_normal-june22/',save_format='tf')
  tf.keras.backend.clear_session()
  gc.collect()

epoch = 0
62/62 [==============================] - 624s 10s/step - loss: 1.5512 - custom_metrics: 0.3584
INFO:tensorflow:Assets written to: /content/drive/MyDrive/new_architecture/model_normal-june22/assets
epoch = 1
62/62 [==============================] - 80s 1s/step - loss: 1.6083 - custom_metrics: 0.3548
epoch = 2
62/62 [==============================] - 80s 1s/step - loss: 1.5541 - custom_metrics: 0.3580
epoch = 3
62/62 [==============================] - 80s 1s/step - loss: 1.5557 - custom_metrics: 0.3637
INFO:tensorflow:Assets written to: /content/drive/MyDrive/new_architecture/model_normal-june22/assets
epoch = 4
62/62 [==============================] - 80s 1s/step - loss: 1.5497 - custom_metrics: 0.3618
epoch = 5
62/62 [==============================] - 80s 1s/step - loss: 1.5329 - custom_metrics: 0.3679
INFO:tensorflow:Assets written to: /content/drive/MyDrive/new_architecture/model_normal-june22/assets
epoch = 6
62/62 [==============================] - 79s 1s/step - loss: 1.5

KeyboardInterrupt: ignored

In [10]:
model=tf.keras.models.load_model('/content/drive/MyDrive/new_architecture/model_normal-june22/',custom_objects={'cross_entropy_loss': cross_entropy_loss ,'dice_coef': dice_coef,'custom_metrics':custom_metrics})
def read_image(path):
    x = cv.imread(path, cv.IMREAD_COLOR)
    x = cv.resize(x,(640,480),interpolation=cv.INTER_AREA)
    x = x/255.0
    x=tf.cast(x,dtype=tf.float32)
    return x

def read_mask(path):
    x = cv.imread(path, cv.IMREAD_GRAYSCALE)
    x = cv.resize(x,(640,480),interpolation=cv.INTER_AREA)
    x = to_categorical(x,num_classes=no_classes+1)
    x = x[:,:,:]
    # x = np.expand_dims(x, axis=-1)
    # x=np.concatenate([x,x],axis=-1)
    x=tf.cast(x,dtype=tf.float32)
    return x
def custom_metrics(y_true,y_pred,smooth=1):
    y_pred = tf.math.argmax(y_pred, axis=-1)
    # y_pred = tf.expand_dims(y_pred,axis=-1)
    y_pred = tf.one_hot(y_pred,38)
    y_true = y_true[:,:,:,1:38]
    y_pred= y_pred[:,:,:,1:38]
    intersection  = tf.math.reduce_sum((y_true*y_pred),axis = [1,2,3])
    union = tf.math.reduce_sum(y_true,axis=[1,2,3])+tf.math.reduce_sum(y_pred,axis = [1,2,3]) - intersection 
    return tf.math.reduce_mean((intersection+smooth)/(union+smooth),axis=0)
def load_data(images,masks):
    length = 1990
    iou=0
    for i,(image,mask) in enumerate(zip(images,masks)):
    
      img = read_image(image)
      mas = read_mask(mask)
      img = np.expand_dims(img,0)
      mas = np.expand_dims(mas,0)
      prediction = model.predict(img)
      iou += custom_metrics(mas,prediction)
      print(i,iou/i)
    return iou/length
no_classes=37
print(len(test_x))
x=load_data(test_x,test_y)
print('*****************************************************************************************************************')
x

1990
0 tf.Tensor(inf, shape=(), dtype=float32)
1 tf.Tensor(1.0947912, shape=(), dtype=float32)
2 tf.Tensor(0.7059876, shape=(), dtype=float32)
3 tf.Tensor(0.68362236, shape=(), dtype=float32)
4 tf.Tensor(0.63011277, shape=(), dtype=float32)
5 tf.Tensor(0.5308399, shape=(), dtype=float32)
6 tf.Tensor(0.56457925, shape=(), dtype=float32)
7 tf.Tensor(0.4839384, shape=(), dtype=float32)
8 tf.Tensor(0.4587962, shape=(), dtype=float32)
9 tf.Tensor(0.44736016, shape=(), dtype=float32)
10 tf.Tensor(0.420333, shape=(), dtype=float32)
11 tf.Tensor(0.4240594, shape=(), dtype=float32)
12 tf.Tensor(0.42458916, shape=(), dtype=float32)
13 tf.Tensor(0.42940283, shape=(), dtype=float32)
14 tf.Tensor(0.42869717, shape=(), dtype=float32)
15 tf.Tensor(0.42177007, shape=(), dtype=float32)
16 tf.Tensor(0.4154273, shape=(), dtype=float32)
17 tf.Tensor(0.44364315, shape=(), dtype=float32)
18 tf.Tensor(0.44063872, shape=(), dtype=float32)
19 tf.Tensor(0.42860374, shape=(), dtype=float32)
20 tf.Tensor(0.449367

<tf.Tensor: shape=(), dtype=float32, numpy=0.3817043>

In [ ]:
model2 = main_model2(shape,no_classes)
model2=tf.keras.models.load_model(filepath2,custom_objects={'cross_entropy_loss':cross_entropy_loss,'dice_coef':dice_coef,'custom_metrics':custom_metrics})

In [ ]:
import matplotlib
cmap = matplotlib.colors.ListedColormap(color_list)
def plot(image,numpy,prediction):
    plt.figure(figsize=(14,14))
    plt.subplot(131)
    plt.imshow(np.squeeze(prediction)+1,cmap=cmap,vmin=1, vmax=len(color_list))
    plt.subplot(132)
    plt.imshow(image)
    plt.subplot(133)
    plt.imshow(numpy,cmap='gray')
    print(prediction)

In [ ]:
import matplotlib
def plot(image,numpy,prediction):
    plt.figure(figsize=(14,14))
    plt.subplot(131)
    plt.imshow(np.squeeze(prediction),cmap=plt.get_cmap('gray'))
    plt.subplot(132)
    plt.imshow(image)
    plt.subplot(133)
    plt.imshow(numpy,cmap='gray')

In [ ]:
def prediction(path):
    image = cv.imread(path,cv.IMREAD_COLOR)
    x = cv.resize(image,(640,480),interpolation=cv.INTER_NEAREST)
    x = x/255.0
    x=np.expand_dims(x,0)
    y = model.predict(x)
    print(y.shape)
    numpy = np.zeros((y.shape[1],y.shape[2],y.shape[3]))
    print(numpy.shape)
    numpy[:,:,:] = y 
    prediction= numpy.argmax(axis=-1)
    print(prediction.shape)
    numpy= tf.one_hot(prediction,38)
    plot(image,numpy[:,:,2],prediction)
    return numpy
path = '/content/drive/MyDrive/depth/image/image1.jpg'
prediction1=prediction(path)

In [ ]:
for i in range(prediction1.shape[2]):
    # for j in range(prediction1.shape[1]):
    #     # if prediction1[i,j]>2:
    (prediction1.shape)
    print(prediction1[])